# RATN 2021: a special cleaning case
Riders started at different days & times in 2021 because of corona. Also, the Timings are notated differently, therefore these should be wrangled apart from the 2018-2020 data. This is done here.

In [359]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
from datetime import timedelta
import calendar
import math
import regex as re

#### Functions

In [443]:
#define the start date & time of a rider
def start(day, time):
    """
    input: day and time as a string (based on column StartTiming)
    
    output: date + time in string format
    """
    # set date and time to empty if the rider did not start (NaN-value)
    if day == "nan":
        date = "1900-1-1"
        time = "0:0" 
        full_date = date + ' ' + time
        return full_date
    #get the day and time of non-NaN values   
    if day == "Fri":
        date = "2021-4-30"
        full_date = date + ' ' + time
        return full_date
    if day == "Sat":
        date = "2021-5-1"
        full_date = date + ' ' + time
        return full_date    

In [444]:
#define the date and time at which the rider passed the checkpoint
def add_time(hours, minutes, startdate):
    """
    input: hours, minutes and startdate as strings. eg hours = "13h" minutes = "04" 
    output: date and time the rider passed a checkpoint as datetime object
    """
    # place "empty" in cells for riders that did not start.
    if startdate  == "1900-1-1 0:0":
        new_time = "empty"
        return new_time
    
    #for everyone who started:
    else:
        #if the Timing is empty (nan), return empty
        if hours == "nan":
            new_time = "empty"
            return new_time
        
        #if there is a valid time in the Timings column, add these to the starttime
        else:
            #hours regex the numbers out of the hours parameter (could contain a 'h')
            hours = re.findall("[\d]+", hours)
            #re returns a list. Get the value and convert it into a integer
            hours = int(hours[0])
                        
            #convert the minutes-string to integer aswell
            minutes = int(minutes)
            
            #input hours and minutes in a time format
            time_change = timedelta(hours=hours, minutes=minutes)
            
            #transform the startdate from string into a datetime object
            startdate = datetime.strptime(startdate, '%Y-%m-%d %H:%M')
            new_time = startdate + time_change
            return new_time

#### Import the data of 2021

In [445]:
ratn_2021 = pd.read_csv('C:/Users/jetsa/Ironhack-project2/data/all_riders_2021.csv')

In [446]:
pd.set_option('display.max_rows', ratn_2021.shape[0]+1)
ratn_2021

,Unnamed: 0,Place,Firstname,Lastname,Gender,Solo or Duo,StartDate,Status,StartTiming,Timing1,Timing2,Timing3,Timing4,Timing5,Timing6,Timing7,Timing8,Timing9,TimingFinish
0,0,1.0,Bas,Vlaskamp,male,Solo,2021,Finisher,Sat 11:00,NaN,13h44m,16h33m,24h01m,30h50m,49h27m,53h30m,68h44m,77h42m,91h49m
1,1,2.0,Sven,Cappuyns,male,Solo,2021,Finisher,Sat 7:45,NaN,14h54m,18h04m,26h49m,33h31m,53h27m,57h26m,76h25m,85h42m,104h28m
2,2,3.0,Richard,Frans,male,Solo,2021,Finisher,Sat 7:15,NaN,17h17m,20h46m,31h24m,46h49m,59h36m,63h45m,85h32m,96h40m,109h57m
3,3,4.0,Paul,Hesen,male,Solo,2021,Finisher,Fri 6:15,05h55m,18h52m,26h31m,35h00m,43h02m,59h16m,62h54m,84h31m,100h00m,112h35m
4,4,5.0,Jason,Schreurs,male,Solo,2021,Finisher,Sat 6:30,05h12m,16h02m,18h51m,32h49m,41h17m,60h07m,65h13m,89h39m,104h46m,117h54m
5,5,6.0,Roeland,van der Hoorn,male,Solo,2021,Finisher,Sat 7:45,05h09m,16h38m,23h17m,33h44m,48h52m,72h10m,78h19m,102h05m,110h23m,123h11m
6,6,7.0,Mark,de Vos,male,Solo,2021,Finisher,Sat 8:00,05h25m,17h30m,22h02m,37h29m,47h27m,62h21m,74h24m,96h50m,107h18m,126h15m
7,7,8.0,Joep,van Luijt,male,Solo,2021,Finisher,Sat 6:45,05h46m,17h54m,20h54m,31h39m,50h19m,63h56m,76h24m,99h50m,108h09m,127h25m
8,8,9.0,Tjerk,Bakker,male,Solo,2021,Finisher,Sat 7:30,05h14m,23h21m,26h45m,36h19m,53h45m,76h51m,82h30m,105h31m,121h56m,132h02m
9,9,10.0,Pier,Nijenhuis,male,Solo,2021,Finisher,Sat 8:00,05h54m,23h21m,27h11m,37h09m,52h48m,77h42m,83h11m,107h27m,122h05m,133h52m


In [447]:
#check types
print(type(ratn_2021["StartTiming"][0])) #finisher
print(type(ratn_2021["StartTiming"][59])) #DNS

<class 'str'>
<class 'float'>


In [448]:
# Transform all cells in the StartTiming column to string (NaN is float)
ratn_2021["StartTiming"] = ratn_2021["StartTiming"].astype(str)

# check a DNS-row to see if it worked
# print(type(ratn_2021["StartTiming"][59])) 

In [450]:
ratn_2021["StartDate"] = ratn_2021.apply(lambda x: start(x["StartTiming"][0:3], x["StartTiming"][4:]), axis = 1)

In [476]:
# ratn_2021

In [452]:
#convert all Timings-columns into strings
ratn_2021[['Timing1', 'Timing2', 'Timing3',
       'Timing4', 'Timing5', 'Timing6', 'Timing7', 'Timing8', 'Timing9',
       'TimingFinish']] = ratn_2021[['Timing1', 'Timing2', 'Timing3',
       'Timing4', 'Timing5', 'Timing6', 'Timing7', 'Timing8', 'Timing9',
       'TimingFinish']].astype(str)

In [475]:
# ratn_2021

In [454]:
#check if it works for 1 column first
#take 3 characters for hours as in the end 100+ hours will be added. We tag along the h, and will get rid of it in the add_time function.
ratn_2021["Checkpoint_1"] = ratn_2021.apply(lambda x: add_time(x["Timing1"][0:3], x["Timing1"][-3:-1], x["StartDate"]), axis = 1)

In [473]:
#check:
# ratn_2021

In [456]:
#apply the function to the other Timing columns
ratn_2021["Checkpoint_2"] = ratn_2021.apply(lambda x: add_time(x["Timing2"][0:3], x["Timing2"][-3:-1], x["StartDate"]), axis = 1)
ratn_2021["Checkpoint_3"] = ratn_2021.apply(lambda x: add_time(x["Timing3"][0:3], x["Timing3"][-3:-1], x["StartDate"]), axis = 1)
ratn_2021["Checkpoint_4"] = ratn_2021.apply(lambda x: add_time(x["Timing4"][0:3], x["Timing4"][-3:-1], x["StartDate"]), axis = 1)
ratn_2021["Checkpoint_5"] = ratn_2021.apply(lambda x: add_time(x["Timing5"][0:3], x["Timing5"][-3:-1], x["StartDate"]), axis = 1)
ratn_2021["Checkpoint_6"] = ratn_2021.apply(lambda x: add_time(x["Timing6"][0:3], x["Timing6"][-3:-1], x["StartDate"]), axis = 1)
ratn_2021["Checkpoint_7"] = ratn_2021.apply(lambda x: add_time(x["Timing7"][0:3], x["Timing7"][-3:-1], x["StartDate"]), axis = 1)
ratn_2021["Checkpoint_8"] = ratn_2021.apply(lambda x: add_time(x["Timing8"][0:3], x["Timing8"][-3:-1], x["StartDate"]), axis = 1)
ratn_2021["Checkpoint_9"] = ratn_2021.apply(lambda x: add_time(x["Timing9"][0:3], x["Timing9"][-3:-1], x["StartDate"]), axis = 1)
ratn_2021["Checkpoint_fin"] = ratn_2021.apply(lambda x: add_time(x["TimingFinish"][0:3], x["TimingFinish"][-3:-1], x["StartDate"]), axis = 1)


In [474]:
#check out the data
# ratn_2021.iloc[:, [1,6, 7, 15, 16,17,18,26, 27, 28]]

In [458]:
# There are DNF with a finish time. This can't be right. Delete the values from the checkpoint_fin column. 
# 5 in total: rows nr: 31,32,33,34,63
# I'll change them by hand

ratn_2021.loc[(ratn_2021["Status"] == "DNF") & (ratn_2021["Unnamed: 0"] == 31), "Checkpoint_fin"] = "empty"
ratn_2021.loc[(ratn_2021["Status"] == "DNF") & (ratn_2021["Unnamed: 0"] == 32), "Checkpoint_fin"] = "empty"
ratn_2021.loc[(ratn_2021["Status"] == "DNF") & (ratn_2021["Unnamed: 0"] == 33), "Checkpoint_fin"] = "empty"
ratn_2021.loc[(ratn_2021["Status"] == "DNF") & (ratn_2021["Unnamed: 0"] == 34), "Checkpoint_fin"] = "empty"
ratn_2021.loc[(ratn_2021["Status"] == "DNF") & (ratn_2021["Unnamed: 0"] == 63), "Checkpoint_fin"] = "empty"

ratn_2021.iloc[[31,32,33,34,63],:] #check it out

,Unnamed: 0,Place,Firstname,Lastname,Gender,Solo or Duo,StartDate,Status,StartTiming,Timing1,...,Checkpoint_1,Checkpoint_2,Checkpoint_3,Checkpoint_4,Checkpoint_5,Checkpoint_6,Checkpoint_7,Checkpoint_8,Checkpoint_9,Checkpoint_fin
31,31,NaN,Jan-Dick,den Das,male,Solo,2021-4-30 10:00,DNF,Fri 10:00,06h16m,...,2021-04-30 16:16:00,2021-05-01 12:43:00,2021-05-01 16:21:00,2021-05-02 12:38:00,2021-05-02 21:26:00,empty,empty,empty,empty,empty
32,32,NaN,Guy,Burgerhout,male,Solo,2021-5-1 6:30,DNF,Sat 6:30,06h03m,...,2021-05-01 12:33:00,2021-05-02 08:50:00,2021-05-02 12:59:00,2021-05-03 16:04:00,empty,empty,empty,empty,empty,empty
33,33,NaN,Wandert,Benthem,male,Solo,2021-4-30 7:15,DNF,Fri 7:15,nan,...,empty,2021-05-01 12:39:00,2021-05-01 16:38:00,2021-05-02 10:15:00,2021-05-02 19:24:00,2021-05-04 12:38:00,2021-05-05 08:44:00,empty,empty,empty
34,34,NaN,Auke (DSQ,Jongbloed,male,Solo,2021-5-1 9:00,DNF,Sat 9:00,05h43m,...,2021-05-01 14:43:00,2021-05-02 12:31:00,2021-05-02 17:44:00,2021-05-03 11:27:00,2021-05-04 11:02:00,2021-05-04 14:02:00,empty,empty,empty,empty
63,63,NaN,NaN,Cullen - Driedonks,male,Duo,2021-5-1 6:15,DNF,Sat 6:15,06h25m,...,2021-05-01 12:40:00,2021-05-02 00:59:00,2021-05-02 04:17:00,2021-05-02 18:46:00,empty,empty,empty,empty,empty,empty


In [460]:
#remove the 1900-01-01 for the DNS. 
ratn_2021.loc[(ratn_2021["Lastname"] == "Mensinga") & (ratn_2021["Unnamed: 0"] == 59), "StartDate"] = ""
ratn_2021.loc[(ratn_2021["Lastname"] == "Heijnen") & (ratn_2021["Unnamed: 0"] == 60), "StartDate"] = ""
# ratn_2021

In [462]:
#turn string into datetime object
ratn_2021['StartDate']= pd.to_datetime(ratn_2021['StartDate'])
# ratn_2021

In [464]:
#remove NaN's from place, change type to int
ratn_2021["Place"]= ratn_2021['Place'].fillna(999)
ratn_2021["Place"]= ratn_2021["Place"].astype(int)

# ratn_2021

In [466]:
#keep the necessary columns and put StartDate right before Checkpoint_1
ratn_2021 = ratn_2021[['Place', 'Firstname', 'Lastname', 'Gender', 'Solo or Duo',
       'Status', 'StartDate','Checkpoint_1', 'Checkpoint_2', 'Checkpoint_3',
       'Checkpoint_4', 'Checkpoint_5', 'Checkpoint_6', 'Checkpoint_7',
       'Checkpoint_8', 'Checkpoint_9', 'Checkpoint_fin']]

In [472]:
ratn_2021

,Place,Firstname,Lastname,Gender,Solo or Duo,Status,StartDate,Checkpoint_1,Checkpoint_2,Checkpoint_3,Checkpoint_4,Checkpoint_5,Checkpoint_6,Checkpoint_7,Checkpoint_8,Checkpoint_9,Checkpoint_fin
0,1,Bas,Vlaskamp,male,Solo,Finisher,2021-05-01 11:00:00,empty,2021-05-02 00:44:00,2021-05-02 03:33:00,2021-05-02 11:01:00,2021-05-02 17:50:00,2021-05-03 12:27:00,2021-05-03 16:30:00,2021-05-04 07:44:00,2021-05-04 16:42:00,2021-05-05 06:49:00
1,2,Sven,Cappuyns,male,Solo,Finisher,2021-05-01 07:45:00,empty,2021-05-01 22:39:00,2021-05-02 01:49:00,2021-05-02 10:34:00,2021-05-02 17:16:00,2021-05-03 13:12:00,2021-05-03 17:11:00,2021-05-04 12:10:00,2021-05-04 21:27:00,2021-05-05 16:13:00
2,3,Richard,Frans,male,Solo,Finisher,2021-05-01 07:15:00,empty,2021-05-02 00:32:00,2021-05-02 04:01:00,2021-05-02 14:39:00,2021-05-03 06:04:00,2021-05-03 18:51:00,2021-05-03 23:00:00,2021-05-04 20:47:00,2021-05-05 07:55:00,2021-05-05 21:12:00
3,4,Paul,Hesen,male,Solo,Finisher,2021-04-30 06:15:00,2021-04-30 12:10:00,2021-05-01 01:07:00,2021-05-01 08:46:00,2021-05-01 17:15:00,2021-05-02 01:17:00,2021-05-02 17:31:00,2021-05-02 21:09:00,2021-05-03 18:46:00,2021-05-04 10:15:00,2021-05-04 22:50:00
4,5,Jason,Schreurs,male,Solo,Finisher,2021-05-01 06:30:00,2021-05-01 11:42:00,2021-05-01 22:32:00,2021-05-02 01:21:00,2021-05-02 15:19:00,2021-05-02 23:47:00,2021-05-03 18:37:00,2021-05-03 23:43:00,2021-05-05 00:09:00,2021-05-05 15:16:00,2021-05-06 04:24:00
5,6,Roeland,van der Hoorn,male,Solo,Finisher,2021-05-01 07:45:00,2021-05-01 12:54:00,2021-05-02 00:23:00,2021-05-02 07:02:00,2021-05-02 17:29:00,2021-05-03 08:37:00,2021-05-04 07:55:00,2021-05-04 14:04:00,2021-05-05 13:50:00,2021-05-05 22:08:00,2021-05-06 10:56:00
6,7,Mark,de Vos,male,Solo,Finisher,2021-05-01 08:00:00,2021-05-01 13:25:00,2021-05-02 01:30:00,2021-05-02 06:02:00,2021-05-02 21:29:00,2021-05-03 07:27:00,2021-05-03 22:21:00,2021-05-04 10:24:00,2021-05-05 08:50:00,2021-05-05 19:18:00,2021-05-06 14:15:00
7,8,Joep,van Luijt,male,Solo,Finisher,2021-05-01 06:45:00,2021-05-01 12:31:00,2021-05-02 00:39:00,2021-05-02 03:39:00,2021-05-02 14:24:00,2021-05-03 09:04:00,2021-05-03 22:41:00,2021-05-04 11:09:00,2021-05-05 10:35:00,2021-05-05 18:54:00,2021-05-06 14:10:00
8,9,Tjerk,Bakker,male,Solo,Finisher,2021-05-01 07:30:00,2021-05-01 12:44:00,2021-05-02 06:51:00,2021-05-02 10:15:00,2021-05-02 19:49:00,2021-05-03 13:15:00,2021-05-04 12:21:00,2021-05-04 18:00:00,2021-05-05 17:01:00,2021-05-06 09:26:00,2021-05-06 19:32:00
9,10,Pier,Nijenhuis,male,Solo,Finisher,2021-05-01 08:00:00,2021-05-01 13:54:00,2021-05-02 07:21:00,2021-05-02 11:11:00,2021-05-02 21:09:00,2021-05-03 12:48:00,2021-05-04 13:42:00,2021-05-04 19:11:00,2021-05-05 19:27:00,2021-05-06 10:05:00,2021-05-06 21:52:00


In [468]:
#write to csv put the year in the name!
ratn_2021.to_csv('all_riders_2021_cleaned.csv')